In [8]:
import os
import random
import cv2
import numpy as np
from shutil import copyfile

# Paths
image_folder = "/media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/dataset/"  # Folder with original images
annotation_folder = "/media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/dataset/"  # Folder with YOLO annotations
augmented_folder = "/media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/augmented_data/"  # Folder to save augmented images and annotations

# Augmentation functions
def add_rain(image):
    noise_intensity = 50
    noise = np.random.randint(0, noise_intensity, image.shape, dtype='uint8')
    rain_image = cv2.add(image, noise)
    return rain_image

def add_fog(image):
    fog_image = cv2.GaussianBlur(image, (5, 5), 0)
    return fog_image

def add_shadow(image):
    shadow = image.copy()
    shadow = cv2.addWeighted(shadow, 0.7, np.zeros(shadow.shape, shadow.dtype), 0, 0)
    return shadow

def augment_image(image, augmentation_type):
    if augmentation_type == "rain":
        return add_rain(image)
    elif augmentation_type == "fog":
        return add_fog(image)
    elif augmentation_type == "shadow":
        return add_shadow(image)

# Randomly select 15% for each weather condition
def randomly_augment_images(image_folder, annotation_folder, augmented_folder):
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    total_images = len(image_files)
    augment_percentage = 0.20  # 15% of total images
    
    # Calculate how many images for each condition
    num_augment = int(total_images * augment_percentage)
    
    # Randomly select images for each weather condition
    rain_images = random.sample(image_files, num_augment)
    fog_images = random.sample([img for img in image_files if img not in rain_images], num_augment)
    shadow_images = random.sample([img for img in image_files if img not in rain_images and img not in fog_images], num_augment)
    
    # Apply augmentations
    for image_list, condition in [(rain_images, 'rain'), (fog_images, 'fog'), (shadow_images, 'shadow')]:
        for image_file in image_list:
            image_path = os.path.join(image_folder, image_file)
            annotation_file = os.path.splitext(image_file)[0] + ".txt"
            annotation_path = os.path.join(annotation_folder, annotation_file)
            
            # Read the image
            image = cv2.imread(image_path)
            if image is None:
                continue  # Skip if image can't be loaded
            
            # Augment the image
            augmented_image = augment_image(image, condition)
            
            # Save augmented image and annotation
            new_image_name = f"{os.path.splitext(image_file)[0]}_{condition}.jpg"
            new_annotation_name = f"{os.path.splitext(annotation_file)[0]}_{condition}.txt"
            
            augmented_image_path = os.path.join(augmented_folder, new_image_name)
            augmented_annotation_path = os.path.join(augmented_folder, new_annotation_name)
            
            cv2.imwrite(augmented_image_path, augmented_image)
            copyfile(annotation_path, augmented_annotation_path)  # Copy the corresponding annotation file

# Example usage
randomly_augment_images(image_folder, annotation_folder, augmented_folder)
